In [1]:
import json   # json: Biblioteca para trabalhar com dados no formato JSON (JavaScript Object Notation).
import os     # os: Biblioteca para interagir com o sistema operacional, como manipulação de arquivos e diretórios.
from datetime import datetime, timedelta  # datetime: Biblioteca para manipular datas e horários.    # timedelta: Classe da biblioteca datetime para realizar operações com intervalos de tempo.
from crewai import Agent, Task, Crew, Process  # crewai: Biblioteca para criar e gerenciar agentes e tarefas em um sistema baseado em IA.  
from crewai_tools import CSVSearchTool   # CSVSearchTool: Ferramenta da biblioteca crewai_tools para realizar buscas em arquivos CSV.
from dotenv import load_dotenv, find_dotenv   # dotenv: Biblioteca para carregar variáveis de ambiente de arquivos .env.
from langchain.tools import Tool   # Tool: Classe base para criar ferramentas personalizadas na biblioteca langchain.
from langchain_community.tools import DuckDuckGoSearchResults   # DuckDuckGoSearchResults: Ferramenta da biblioteca langchain_community para realizar buscas no DuckDuckGo.

c:\Users\PC\Downloads\CrewAI_imobiliaria\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())  # Carrega variáveis de ambiente do arquivo .env.
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")  # Configura o modelo de linguagem GPT-3.5-turbo para ser usado como LLM.

In [3]:
csv_imoveis = CSVSearchTool(csv="files/imoveis.csv")  # Ferramenta para buscar dados no arquivo CSV de imóveis.

In [4]:
# Agente Corretor de Imóveis
corretor_imoveis = Agent(
    role="Corretor de Imóveis",
    goal="Obtenha as preferências do cliente e busque imóveis compatíveis no banco de dados",
    backstory="Especialista no mercado imobiliário, encontra as melhores opções baseadas no perfil do cliente",
    verbose=True,
    max_iter=5,
    tools=[csv_imoveis],
    allow_delegation=False,
    memory=True
)

In [5]:
# Tarefa - Buscar Imóveis
buscar_imoveis = Task(
    description="Pesquise imóveis na região desejada pelo cliente, considerando faixa de preço e tipo de imóvel",
    expected_output="Lista de imóveis disponíveis com detalhes sobre localização, preço e características",
    agent=corretor_imoveis
)

In [6]:
from crewai.tools import BaseTool

def obter_precos_imoveis(cidade: str = "geral"):
    # Função que retorna a tendência de preços de imóveis com base na cidade especificada.
    # Se a cidade não for encontrada no dicionário, retorna os valores padrão definidos para "geral".
    precos = {
        "São Paulo": {"tendencia":"aumento", "percentual":5.2},  # Tendência de aumento de 5.2% em São Paulo.
        "Rio de Janeiro": {"tendencia":"estavel", "percentual": 0.0},  # Tendência estável no Rio de Janeiro.
        "Belo Horizonte": {"tendencia":"queda", "percentual":-3.1},  # Tendência de queda de 3.1% em Belo Horizonte.
        "geral": {"tendencia":"aumento", "percentual":4.0}  # Tendência geral de aumento de 4.0%.
    }
    return precos.get(cidade, precos["geral"])  # Retorna os dados da cidade ou os valores gerais.

class TendenciaPrecosImoveisTool(BaseTool):
    # Classe que define uma ferramenta personalizada para analisar tendências de preços de imóveis.
    name: str = "Analisador de Preços Imobiliários"  # Nome da ferramenta.
    description: str = "Obtém tendências de preços de imóveis com base na cidade especificada."  # Descrição da ferramenta.

    def _run(self, cidade: str) -> dict:
        """
        Método principal que executa a análise de preços imobiliários.
        Recebe o nome da cidade como parâmetro e retorna a tendência de preços.
        """
        try:
            return obter_precos_imoveis(cidade)  # Chama a função para obter os preços da cidade.
        except Exception as e:
            # Em caso de erro, retorna um dicionário com a mensagem de erro.
            return {"erro": f"Erro ao obter tendências de preços {str(e)}"}
        

In [7]:
# Cria um agente chamado "Analista de Mercado Imobiliário", que analisa tendências de preços e faz previsões com base em dados históricos.
analista_mercado = Agent(
    role="Analista de Mercado Imobiliário",
    goal="Analisa tendências de preços e ajuda a prever a valorização ou desvalorização dos imóveis na cidade {cidade}",
    backstory="Experiente no setor, usa dados históricos para prever preços futuros.",
    verbose=True,
    max_iter=5,
    allow_delegation=False,
    memory=True,
)

In [8]:
# Define uma tarefa para o agente "Analista de Mercado Imobiliário", que analisa tendências de preços com base na cidade e outros parâmetros.
obter_tendencias = Task(
    description="""
    Analise o histórico de preços de imóveis na cidade {cidade} e forneça insights sobre
    valorização ou desvalorização. Considere o tipo de imóvel {tipo_imovel} e a 
    faixa de preço {faixa_preco}
    """,
    expected_output="Resumo da tendência dos preços no mercado imobiliário",
    tools=[TendenciaPrecosImoveisTool()],
    agent=analista_mercado,
    parameters=["cidade"]
)

In [9]:
# Cria um agente chamado "Analista de Notícias Imobiliários", que busca e analisa notícias relevantes sobre o mercado imobiliário.
analista_noticias = Agent(
    role="Analista de Notícias Imobiliários",
    goal="Busca notícias relevantes sobre o mercado imobiliário para avaliar fatores externos.",
    backstory="Especialista em analisar notícias e tendências econômicas que afetam os preços dos imóveis.",
    verbose=True,
    max_iter=5,
    memory=True,
)

In [10]:
# Configura uma ferramenta para buscar notícias no DuckDuckGo, retornando até 5 resultados.
searchTool = DuckDuckGoSearchResults(backend="news", num_results=5)

In [11]:
searchTool

DuckDuckGoSearchResults(max_results=5, api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'), backend='news')

In [12]:
# Define uma tarefa para o agente "Analista de Notícias Imobiliários", que busca notícias recentes sobre o mercado imobiliário.
buscar_noticias = Task(
    description=f"Pesquise notícias recentes sobre o mercado imobiliário. Data atual: {datetime.now()}",
    expected_output="Resumo das principais notícias e tendências imobiliárias.",
    agent=analista_noticias,
    tool=[searchTool],
)

In [13]:
# Cria um agente chamado "Consultor Financeiro", que analisa opções de financiamento imobiliário.
consultor_financeiro = Agent(
    role="Consultor Financeiro",
    goal="Analisa opções de financiamento imobiliário com base no perfil do cliente.",
    backstory="Especialista em crédito imobiliário, ajuda clientes a escolherem as melhores opções de financiamento",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
)

In [14]:
# Define uma tarefa para o agente "Consultor Financeiro", que sugere opções de financiamento com base na renda do cliente.
calcular_financiamento = Task(
    description="Analise a renda do cliente e sugira opções de financiamento viáveis.",
    expected_output="Tabela comparativa com diferentes financiamentos, taxa de juros e prazos",
    agent=consultor_financeiro,
)

In [15]:
# Cria um agente chamado "Redator de Relatórios Imobiliários", que gera relatórios detalhados e persuasivos.
redator = Agent(
    role="Redator de Relatórios Imobiliários",
    goal="Gera um relatório em Markdown completo e persuasivo com base nas análises de mercado e imóveis encontrados.",
    backstory="Especialista em comunicação, traduz dados complexos para clientes de forma clara e objetiva.",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
)

In [16]:
# Define uma tarefa para o agente "Redator", que gera um relatório detalhado com base nas análises realizadas.
gerar_relatorio = Task(
    description="Gere um relatório detalhado sobre o melhor imóvel encontrado, considerando preços, tendências e financiamento.",
    expected_output="Um relatório completo e persuasivo em formato Markdown, com resumo do mercado, opções recomendadas e justificativa da escolha.",
    agent=redator,
    context=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financiamento],
)


In [17]:
# Configura a equipe (`Crew`) com os agentes e tarefas definidos, utilizando um processo hierárquico para gerenciar as interações.
crew = Crew(
    agents=[corretor_imoveis, analista_mercado, analista_noticias, consultor_financeiro, redator],
    tasks=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financiamento, gerar_relatorio],
    verbose=True,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    max_iter=15,
    manager_llm=llm,
)

In [21]:
# Inicia o processo da equipe com os parâmetros fornecidos, como cidade, tipo de imóvel e faixa de preço.
result = crew.kickoff(inputs={"cidade": "Rio de Janeiro",
                              "tipo_imovel": "Apartamento",
                              "faixa_preco": "500000-700000"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ef39483a-3b09-4630-ae26-a2b2c0e713b5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 52983914-b626-4393-9dcd-d4f0c8dd5500
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 52983914-b626-4393-9dcd-d4f0c8dd5500
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress

# Agent: Crew Manager
## Task: Pesquise imóveis na região desejada pelo cliente, considerando faixa de preço e tipo de imóvel


🤖 Agent: Crew Manager
    Status: In Progress
└── 🧠 Thinking...

...



# Agent: Crew Manager
## Final Answer: 
The detailed report on the best property found, considering prices, trends, and financing options has been successfully delegated to the Redator de Relatórios Imobiliários. The report includes a thorough analysis of the property ID 20, located at Rua das Palmeiras, 123 in São Paulo, with insights on market trends, pricing, and financing recommendations.




🚀 Crew: crew
├── 📋 Task: 52983914-b626-4393-9dcd-d4f0c8dd5500
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
├── 📋 Task: 39ade002-e8f9-4914-8f35-54698e363793
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
├── 📋 Task: 2e93516a-a6a0-45f7-ba87-6ff78fe38bc8
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
├── 📋 Task: 14f9ca87-b01c-4020-8fa2-7446a4e16c2a
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
└── 📋 Task: 37cf11f6-7b59-4f08-81f2-a2c220fe6850
       Status: Executing Task...
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    └── 🤖 Agent: Crew Manager
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 52983914-b626-4393-9dcd-d4f0c8dd5500
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
├── 📋 Task: 39ade002-e8f9-4914-8f35-54698e363793
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
├── 📋 Task: 2e93516a-a6a0-45f7-ba87-6ff78fe38bc8
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
├── 📋 Task: 14f9ca87-b01c-4020-8fa2-7446a4e16c2a
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
└── 📋 Task: 37cf11f6-7b59-4f08-81f2-a2c220fe6850
       Assigned to: Crew Manager
       Status: ✅ Completed
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    └── 🤖 Agent: Crew Manager
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 37cf11f6-7b59-4f08-81f2-a2c220fe6850                                                                     │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ef39483a-3b09-4630-ae26-a2b2c0e713b5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [22]:
# Importa funções para exibir o resultado no formato Markdown no Jupyter Notebook.
from IPython.display import display, Markdown

In [23]:
# Exibe o resultado do processo da equipe em formato Markdown para facilitar a leitura.
display(Markdown(str(result)))

The detailed report on the best property found, considering prices, trends, and financing options has been successfully delegated to the Redator de Relatórios Imobiliários. The report includes a thorough analysis of the property ID 20, located at Rua das Palmeiras, 123 in São Paulo, with insights on market trends, pricing, and financing recommendations.